# Image Classification
supervised machine learning algorithm used for classification tasks. It works by splitting the dataset into smaller and smaller subsets based on feature values, forming a tree-like structure of decisions. Each internal node represents a decision rule (based on a feature), each branch represents an outcome of that decision, and each leaf node represents a final class (prediction).

In [3]:
!pip install split-folders

In [7]:
!pip install tensorflow

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.32.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached wrapt-1.17.3-cp313-cp313-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached kera

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\Users\\786\\Desktop\\Sessions\\MachineLearning & DeepLearning\\07-Image-Classification-Tensorflow-DL\\.venv\\Lib\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\lib\\security\\credentials\\gcp_service_account_identity\\gcp_service_account_identity_credentials.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [5]:
import splitfolders
import os

# Define the input and output directories
input_folder = "Rice_Image_Dataset"
output_folder = "rice_data_split" # The new folder where train/val/test will be created

# Specify the ratio (e.g., 80% train, 20% validation, 0% test)
# The sum must be 1.0 (or 100).
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.2, 0.0))

print(f"Data split successfully into {output_folder}")

Data split successfully into rice_data_split


In [2]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory


# Define parameters
image_size = (250, 250) # Standard size for many models
batch_size = 32
seed = 42
data_dir = "./Dataset"

# Create the training dataset (e.g., 80% of the data)
train_ds = image_dataset_from_directory(
    data_dir+"/train", # Pointing to the training subfolder
    validation_split=0.2, # 20% reserved for validation
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

# Create the validation dataset (the remaining 20%)
val_ds = image_dataset_from_directory(
    data_dir+"/test", # Pointing to the validation subfolder
    validation_split=0.2, # Must be the same value as above
    subset="validation",
    seed=seed, # Must be the same seed as above to ensure non-overlap
    image_size=image_size,
    batch_size=batch_size
)

# Print the class names (should be your 5 folder names)
class_names = train_ds.class_names
print(f"Found class names: {class_names}")

# You can now pass train_ds and val_ds directly to model.fit()

Found 557 files belonging to 2 classes.
Using 446 files for training.
Found 140 files belonging to 2 classes.
Using 28 files for validation.
Found class names: ['cats', 'dogs']


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape (based on your image_dataset_from_directory setup)
INPUT_SHAPE = (250, 250, 3) # Height, Width, Color Channels (RGB)
NUM_CLASSES = 2            # You have 5 subfolders/classes

In [15]:
model = models.Sequential([
    # Input Layer
    layers.Input(shape=INPUT_SHAPE),

    # Block 1: Feature Extraction
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Block 2: Feature Extraction
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Block 3: Feature Extraction
    layers.Conv2D(64, (3, 3), activation='relu'),

    # Flatten the 3D output to 1D for the Dense layers
    layers.Flatten(),

    # Fully Connected (Dense) Layers for Classification
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5), # Add dropout for regularization
    layers.Dense(NUM_CLASSES, activation='softmax') # Output layer
])

# Display the model structure
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 248, 248, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 59, 59, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 222784)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │    14,258,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,314,690 (54.61 MB)

 Trainable params: 14,314,690 (54.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [19]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# 1. Define the directory to save the checkpoints
checkpoint_dir = './training_checkpoints_cnn'
os.makedirs(checkpoint_dir, exist_ok=True) # Create the folder if it doesn't exist

# 2. Define the filepath for the model files
# The format includes epoch number and the validation loss/accuracy
checkpoint_filepath = os.path.join(
    checkpoint_dir,
    'pet_cnn_model_{epoch:02d}-{val_accuracy:.4f}.keras'
)

# 3. Create the ModelCheckpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,   
    save_weights_only=False,       # Set to True if you only want to save weights
    monitor='val_accuracy',        # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
    mode='max',                    # 'max' means we look for the highest val_accuracy
    save_best_only=True,           # Only save a model if 'val_accuracy' improves
    verbose=1                      # Log when a new best model is saved
)

In [20]:
EPOCHS = 10

print("Starting model training with Checkpoints...")

# List of callbacks to pass to fit()
callbacks_list = [
    model_checkpoint_callback,
]

history = model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=EPOCHS,
    callbacks=callbacks_list 
)

print("Training finished.")

# The best model will be saved in the 'training_checkpoints' folder.

Starting model training with Checkpoints...
Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9266 - loss: 0.2115
Epoch 1: val_accuracy improved from None to 0.46429, saving model to ./training_checkpoints_cnn\pet_cnn_model_01-0.4643.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9238 - loss: 0.3139 - val_accuracy: 0.4643 - val_loss: 1.5717
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8903 - loss: 0.1990
Epoch 2: val_accuracy improved from 0.46429 to 0.50000, saving model to ./training_checkpoints_cnn\pet_cnn_model_02-0.5000.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8789 - loss: 0.2362 - val_accuracy: 0.5000 - val_loss: 0.9980
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9275 - loss: 0.2121
Epoch 3: val_accuracy did not improve from 0.50000
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9417 - loss: 0.1710 - val_accuracy: 0.4286 - val_loss: 2.3502
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accur

In [22]:
import tensorflow as tf
import numpy as np

# Path to your image
image_path = "sample-dog-1.jpg"

# Must match the size you used during training
img_height, img_width = 250, 250  

# 1. Load the image
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))

# 2. Convert image to array
img_array = tf.keras.preprocessing.image.img_to_array(img)

# 3. Add batch dimension (model expects batch of images)
img_array = tf.expand_dims(img_array, 0)  # Shape becomes (1, 250, 250, 3)

# 4. Normalize (if your dataset wasn’t normalized automatically)
img_array = img_array / 255.0  

# 5. Make prediction
predictions = model.predict(img_array)
print(predictions)  # This will show the raw prediction scores
predicted_class_index = np.argmax(predictions[0])

# 6. Map index to class name
class_names = train_ds.class_names
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")
print(f"Confidence: {np.max(predictions[0]):.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
[[0.4827 0.5173]]
Predicted class: dogs
Confidence: 0.52


In [24]:
import tensorflow as tf
import numpy as np

# Path to your image
image_path = "sampl-cat.jpg"

# Must match the size you used during training
img_height, img_width = 250, 250  

# 1. Load the image
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))

# 2. Convert image to array
img_array = tf.keras.preprocessing.image.img_to_array(img)

# 3. Add batch dimension (model expects batch of images)
img_array = tf.expand_dims(img_array, 0)  # Shape becomes (1, 250, 250, 3)

# 4. Normalize (if your dataset wasn’t normalized automatically)
img_array = img_array / 255.0  

# 5. Make prediction
predictions = model.predict(img_array)
print(predictions)  # This will show the raw prediction scores
predicted_class_index = np.argmax(predictions[0])

# 6. Map index to class name
class_names = train_ds.class_names
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")
print(f"Confidence: {np.max(predictions[0]):.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
[[0.52001137 0.47998866]]
Predicted class: cats
Confidence: 0.52
